## Analysis with Display

In [ ]:
import pandas as pd
import numpy as np
import requests
from keras.models import load_model
from keras.preprocessing import image
import pymysql as sql
from selenium import webdriver
from PIL import Image
import os
import re

In [ ]:
df=pd.read_csv('hist_data.csv', encoding="cp1252")
model = load_model('nsfw.h5')
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
os.system('cls')

In [ ]:
def rateurl(i):
            BASE = 'https://render-tron.appspot.com/screenshot/'
            path = 'target.jpg'
            response = requests.get(BASE + i, stream=True)
            if response.status_code != 200:
                selfie(i)
            if response.status_code == 200:
                with open(path, 'wb') as file:
                    for chunk in response:
                        file.write(chunk)
            test_image = image.load_img('target.jpg', target_size = (64, 64)) 
            test_image = image.img_to_array(test_image)
            test_image = np.expand_dims(test_image, axis = 0)
            result = model.predict(test_image)
            categories=['safe','not safe']
            status=categories[int(result[0][0])]
            return status

        
def selfie(url):
    DRIVER = 'chromedriver'
    driver = webdriver.Chrome(DRIVER)
    valid=re.match(regex,url) is not None
    driver.get(url)
    screenshot = driver.save_screenshot('target.png')
    driver.quit()
    im = Image.open("target.png")
    rgb_im = im.convert('RGB')
    rgb_im.save('colors.jpg')

In [ ]:
def statuscheck(url):
    c=sql.connect(host="localhost",user="root",password="",db="urls")
    connection=c.cursor()
    command="SELECT status FROM urls.url_status where name='"+ url +"';"
    connection.execute(command)
    retrived=connection.fetchall()
    return retrived

def urlinsert(url):
    c=sql.connect(host="localhost",user="root",password="",db="urls")
    connection=c.cursor()
    status=rateurl(url)
    if len(url) > 255:
        print(url)
    command="insert into urls.url_status values('" +url+ "','" +status+ "');"
    connection.execute(command)
    return status

In [ ]:
def sync(url):
    status=statuscheck(url)
    if len(status)!=0:
        return status
    else:
        return urlinsert(url)

In [ ]:
def result(url):
    status=str(url)
    status=status.split()
    status=status[0]
    status=sync(status)
    if type(status)==str:
        return status
    if type(status)==tuple:
        return status[0][0]

In [ ]:
for i in range(100):
    print(i,result(df.iloc[[i],[0]].values[0][0]))




In [ ]:
dataframe=pd.DataFrame(urlscan)
dataframe